In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score

In [2]:
df_main_train = pd.read_csv(
    "assignment_2_train.csv"
)

df_leader_board = pd.read_csv(
    "assignment_2_test.csv"
)

print("df_main_train.shape = {} rows, {} cols".format(*df_main_train.shape))
#print("df_valid.shape = {} rows, {} cols".format(*df_valid.shape))
print("df_leader_board.shape = {} rows, {} cols".format(*df_leader_board.shape))
df_main_train.head(n=5)

df_main_train.shape = 180000 rows, 394 cols
df_leader_board.shape = 100001 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_main_train.drop(['TransactionDT', 'TransactionID'], axis='columns', inplace=True)
df_leader_board.drop(['TransactionDT', 'TransactionID'], axis='columns', inplace=True)

In [4]:
y_main_train = df_main_train['isFraud'].values
y_leader_board = df_leader_board['isFraud'].values

#df_main_train.drop(['isFraud'], axis='columns', inplace=True)
df_leader_board.drop(['isFraud'], axis='columns', inplace=True)

In [5]:
numeric_feature = df_main_train.select_dtypes(include=[np.number]).columns

сategorical_feature = list(set(df_main_train.columns) - set(numeric_feature))

df_main_train_dummy = df_main_train.copy()
#df_valid_dummy = df_valid.copy()
df_leader_board_dummy = df_leader_board.copy()

for i in range(len(сategorical_feature)):
    print(f'feature {сategorical_feature[i]}')
    сategorical_train = df_main_train[сategorical_feature[i]].unique()
    #сategorical_valid = df_valid[сategorical_feature[i]].unique()
    сategorical_test = df_leader_board[сategorical_feature[i]].unique()
    #print(f'сategorical train {сategorical_train}')
    #print(f'сategorical test {сategorical_test}')
    
    #cat_in_feature = list(set(сategorical_train) & set(сategorical_valid) & set(сategorical_test))
    cat_in_feature = list(set(сategorical_train) & set(сategorical_test))
    for cat in cat_in_feature:
        dummie_feature_name = f'{сategorical_feature[i]}_{cat}'
        #print(dummie_feature_name)
        
        if str(cat) != 'nan':
            print(f'    dummie feature name: {dummie_feature_name}')
            df_main_train_dummy[f'{сategorical_feature[i]}_{cat}'] = 0
            df_main_train_dummy.loc[df_main_train_dummy.loc[:,сategorical_feature[i]]==cat, 
                                                          [dummie_feature_name]] = 1      
            
            #df_valid_dummy[f'{сategorical_feature[i]}_{cat}'] = 0
            #df_valid_dummy.loc[df_valid_dummy.loc[:,сategorical_feature[i]]==cat, 
            #                                              [dummie_feature_name]] = 1  
            
            df_leader_board_dummy[f'{сategorical_feature[i]}_{cat}'] = 0
            df_leader_board_dummy.loc[df_leader_board_dummy.loc[:,сategorical_feature[i]]==cat, 
                                                          [dummie_feature_name]] = 1 
            
    df_main_train_dummy.drop([сategorical_feature[i]], axis='columns', inplace=True)
    #df_valid_dummy.drop([сategorical_feature[i]], axis='columns', inplace=True)
    df_leader_board_dummy.drop([сategorical_feature[i]], axis='columns', inplace=True)
    
print(f"count of categorical_df_train {df_main_train_dummy.shape}")
#print(f"count of categorical_df_valid {df_valid_dummy.shape}")
print(f"count of categorical_df_test {df_leader_board_dummy.shape}")

feature M2
    dummie feature name: M2_T
    dummie feature name: M2_F
feature M4
    dummie feature name: M4_M0
    dummie feature name: M4_M2
    dummie feature name: M4_M1
feature M9
    dummie feature name: M9_T
    dummie feature name: M9_F
feature M3
    dummie feature name: M3_F
    dummie feature name: M3_T
feature P_emaildomain
    dummie feature name: P_emaildomain_me.com
    dummie feature name: P_emaildomain_live.fr
    dummie feature name: P_emaildomain_aol.com
    dummie feature name: P_emaildomain_optonline.net
    dummie feature name: P_emaildomain_web.de
    dummie feature name: P_emaildomain_verizon.net
    dummie feature name: P_emaildomain_yahoo.de
    dummie feature name: P_emaildomain_hotmail.com
    dummie feature name: P_emaildomain_cableone.net
    dummie feature name: P_emaildomain_mac.com
    dummie feature name: P_emaildomain_juno.com
    dummie feature name: P_emaildomain_gmail.com
    dummie feature name: P_emaildomain_hotmail.de
    dummie feature name: P

In [6]:
from sklearn.model_selection import TimeSeriesSplit

In [7]:
n_splits = 5

tscv = TimeSeriesSplit(max_train_size=100000, n_splits=5)
print(tscv)

TimeSeriesSplit(max_train_size=100000, n_splits=5)


In [8]:
df_train = df_main_train_dummy.copy()

y_train_ = df_train['isFraud']
df_train.drop(['isFraud'], axis='columns', inplace=True)

print("df_train.shape = {} rows, {} cols".format(*df_train.shape))

df_train.shape = 180000 rows, 524 cols


In [9]:
params = {
    "learning_rate": 0.1,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 5,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

model = xgb.XGBRegressor(params=params)

In [10]:
for train_index, test_index in tscv.split(df_main_train):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = df_train.iloc[train_index, :], df_train.iloc[test_index, :]
    y_train, y_test = y_train_[train_index], y_train_[test_index]
    model.fit(X_train, y_train,
             eval_set=[(X_train, y_train), (X_test, y_test)],
             early_stopping_rounds=20,
             eval_metric=['auc'],
             verbose=10)
    print("="*69)
    

TRAIN: 30000 TEST: 30000
[14:00:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-auc:0.616162	validation_1-auc:0.658669
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 20 rounds.
[10]	validation_0-auc:0.800807	validation_1-auc:0.801967
[20]	validation_0-auc:0.84358	validation_1-auc:0.835569
[30]	validation_0-auc:0.861104	validation_1-auc:0.844259
[40]	validation_0-auc:0.867954	validation_1-auc:0.848469
[50]	validation_0-auc:0.872632	validation_1-auc:0.853604
[60]	validation_0-auc:0.880655	validation_1-auc:0.855949
[70]	validation_0-auc:0.885002	validation_1-auc:0.859319
[80]	validation_0-auc:0.88878	validation_1-auc:0.861613
[90]	validation_0-auc:0.892416	validation_1-auc:0.863962
[99]	validation_0-auc:0.894295	validation_1-auc:0.865437
TRAIN: 60000 TEST: 30000
[14:00:41] WARNING: src/objective/regression_obj.cu:1

In [13]:
predict = model.predict(df_leader_board_dummy)
print(f'leader_board: {round(roc_auc_score(y_leader_board, predict), 4)}')

leader_board: 0.8402


In [ ]:
# Почемуто получилось, что CV не способствует повышению качаства на ЛБ ланных.

Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).

In [ ]:
# Не представляю как.